In [ ]:
import time  
import requests
import json
import re
import urllib.request as request
from hashlib import md5

In [ ]:
def writeLog(text):
    fh = open('Log.txt', 'a')
    fh.write(text+'\n')
    fh.close()

In [ ]:
# 若快打码，用于验证码识别
class RClient(object):
    def __init__(self, username, password, soft_id, soft_key):
        self.username = username
        self.password = md5(password.encode("utf8")).hexdigest()
        self.soft_id = soft_id
        self.soft_key = soft_key
        self.base_params = {
            'username': self.username,
            'password': self.password,
            'softid': self.soft_id,
            'softkey': self.soft_key,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'Expect': '100-continue',
            'User-Agent': 'ben',
        }
    def rk_create(self, im, im_type, timeout=60):
        """
        im: 图片字节
        im_type: 题目类型
        """
        params = {
            'typeid': im_type,
            'timeout': timeout,
        }
        params.update(self.base_params)
        files = {'image': ('a.jpg', im)}
        r = requests.post('http://api.ruokuai.com/create.json', data=params, files=files, headers=self.headers)
        return r.json()
    def rk_report_error(self, im_id):
        """
        im_id:报错题目的ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://api.ruokuai.com/reporterror.json', data=params, headers=self.headers)
        return r.json()

In [ ]:
# 获取登陆用图形验证码，并识别返回
def getCode(session, rkAccount, rkPassword): 
    headers = {'User-Agent':'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore',
               'Host': 'login.360.cn',
               'Referer': 'http://i.360.cn/login/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F%23%2Fhome'}
    Time = str(int(time.time() * 1000))
    data = {'src': 'pcw_i360',
            'o': 'sso',
            'm': 'getCaptcha',
            'app': 'i360',
            'scene': 'login',
            'callback': 'QiUserJsonP'+Time
           }
    codedata = session.get('http://login.360.cn/?src=pcw_i360&o=sso&m=getCaptcha&app=i360&scene=login&callback=QiUserJsonP', data=data, headers=headers).content.decode(encoding='UTF8')
    codedatadict = json.loads(re.findall(r"\((.*)\)", codedata)[0])
    codeToken = codedatadict.get('crypt_code')
    codeImgUrl = codedatadict.get('img')
    request.urlretrieve(codeImgUrl, './code.jpg')
    rc = RClient(rkAccount, rkPassword, '106432', 'b46894ebdead4d4eb3c6703b58da5b24')
    im = open('./code.jpg', 'rb').read()
    codeResult = rc.rk_create(im, 3000)['Result']
    print("识别验证码为 "+codeResult)
    writeLog("识别验证码为 "+codeResult)
    return codeToken, codeResult

In [ ]:
# 获取登陆用Token
def getToken(session, account): 
    headers = {'User-Agent':'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore',
               'Host': 'login.360.cn',
               'Referer': 'http://i.360.cn/login/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F%23%2Fhome'}
    Time = str(int(time.time() * 1000))
    data = {'o': 'sso',
            'm': 'getToken',
            'func': 'QHPass.loginUtils.tokenCallback',
            'userName': account,
            'rand': '0.27856458922300353',
            'quc_lang': '',
            'callback': 'QiUserJsonP'+Time
           }
    tokendata = session.get('https://login.360.cn/?o=sso&m=getToken&func=QHPass.loginUtils.tokenCallback&userName='+account+'&rand=0.27856458922300353&quc_lang=&callback=QiUserJsonP'+Time, data=data, headers=headers).content.decode(encoding='UTF8')
    tokendatadict = json.loads(re.findall(r"\((.*)\)", tokendata)[0])
    token = tokendatadict.get('token')
    return token

In [ ]:
# 登陆
def login(session, account, password, codeToken, codeResult, token, count):
    count+=1
    headers = {'User-Agent':'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore',
               'Host': 'login.360.cn',
               'Referer': 'http://i.360.cn/login/wap?src=mpw_blockcat&destUrl=http%3A%2F%2Fblockcat.360.cn%2F%23%2Fhome'}
    Time = str(int(time.time() * 1000))
    HD5password  = md5(password.encode("utf8")).hexdigest()
    data = {'func': 'jQuery1124007363845979828043_1529113215170',
            'userName': account, 
            'pwdmethod': '1',
            'password': HD5password,
            'isKeepAlive': '1',
            'token': token,
            'r': Time,
            'captFlag': '1',
            'captchaApp': 'i360',
            'captcha': codeResult, 
            'o': 'sso',
            'm': 'login',
            'from': 'mpw_blockcat',
            'rtype': 'data',
            'requestScema': 'http',
            'quc_lang': '',
            'qucrypt_code': codeToken,
            '_': Time,
           }
    logindata = session.get('https://login.360.cn/?func=jQuery1124007363845979828043_1529113215170&userName='+account+'&pwdmethod=1&password='+HD5password+'&isKeepAlive=1&token='+token+'&r='+Time+'&captFlag=1&captchaApp=i360&captcha='+codeResult+'&o=sso&m=login&from=mpw_blockcat&rtype=data&requestScema=http&quc_lang=&qucrypt_code='+codeToken+'&_='+Time, data=data, headers=headers).content.decode(encoding='UTF8')
    logindatadict = json.loads(re.findall(r"\((.*)\)", logindata)[0])
    if logindatadict.get('errno')==0:
        print("登陆成功！")
        writeLog("登陆成功！")
        fh = open('登陆成功.txt', 'a')
        fh.write(account+"----"+password+"\n")
        fh.close()
    elif logindatadict.get('errno')==78000 and count<3:
        print("登陆失败,验证码错误，尝试第"+str(count+1)+"次")
        writeLog("登陆失败,验证码错误，尝试第"+str(count+1)+"次")
        codeToken, codeResult = getCode(session, rkAccount, rkPassword)
        token = getToken(session, account)
        login(session, account, password, codeToken, codeResult, token, count)
        
    else:
        print("登陆失败\n错误信息:"+logindatadict.get('errmsg').encode().decode("UTF8"))
        writeLog("登陆失败\n错误信息:"+logindatadict.get('errmsg').encode().decode("UTF8"))
        fh = open('登陆失败.txt', 'a')
        fh.write(account+"----"+password+"----错误信息:"+logindatadict.get('errmsg').encode().decode("UTF8")+"\n")
        fh.close()
    return session

In [ ]:
# 领取鱼币
def getGold(session, ID):
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'api.blockcat.360.cn',
               'Referer': 'http://blockcat.360.cn/',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    data = {'id': ID, '__did': '', '__src': '', 'callback': 'helpers.JsonpReq.completeCall.call_1'}
    redata = session.get('https://api.blockcat.360.cn/kitty/cat/clear?id='+str(ID)+'&__did=&__src=&callback=helpers.JsonpReq.completeCall.call_1', data=data, headers=headers).content.decode(encoding='UTF8')
    datadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if datadict.get('errno')==0:
        print("ID:"+str(ID)+" 领取鱼币成功！")
        writeLog("ID:"+str(ID)+" 领取鱼币成功！")
    else:
        print("ID:"+str(ID)+" 领取鱼币失败")
        writeLog("ID:"+str(ID)+" 领取鱼币失败")

In [ ]:
# 扫描账号是否有鱼币可领取
def getData(account, password, session):
    code=0
    headers = {'Accept' : '*/*',
               'Accept-Language' : 'zh-CN,zh;q=0.8',
               'Connection': 'keep-alive',
               'Host': 'api.blockcat.360.cn',
               'Referer': 'http://blockcat.360.cn/',
               'User-Agent': 'Mozilla/5.0 (Linux; U; Android 4.4.4; en-US; XT1022 Build/KXC21.5-40) AppleWebKit/534.30 (KHTML, like Gecko) Version/4.0 UCBrowser/10.7.0.636 U3/0.8.0 Mobile Safari/534.30;360appstore'}
    data = {'__did': '', '__src': '', 'callback': 'helpers.JsonpReq.completeCall.call_1'}
    redata = session.get('https://api.blockcat.360.cn/kitty/cat/events?__did=&__src=&callback=helpers.JsonpReq.completeCall.call_1', data=data, headers=headers).content.decode(encoding='UTF8')
    datadict = json.loads(re.findall(r"\((.*)\)", redata)[0])
    if datadict.get('errno')==0:
        if type(datadict.get('data')) is dict:
            catList = []
            for i in ['e1', 'e2', 'e3']:
                catDict = datadict.get('data').get(i)
                if catDict:
                    for i in catDict:
                        getGold(session, i)
        else:
            print("暂无鱼币可领取")
            writeLog("暂无鱼币可领取")
            fh = open('领取失败.txt', 'a')
            fh.write(account+"----"+password+"----暂无鱼币可领取\n")
            fh.close()
    else:
        code=1
        print("领取错误")
        writeLog("领取错误")
        fh = open('领取失败.txt', 'a')
        fh.write(account+"----"+password+"----领取错误\n")
        fh.close()
    return code

In [ ]:
def mainControl(account, password):
    print("-----------------正在执行账号"+str(account)+"-----------------\nTime:"+
          str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
    writeLog("-----------------正在执行账号"+str(account)+"-----------------\nTime:"+
          str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime())))
    with requests.session() as session:
        codeToken, codeResult = getCode(session, rkAccount, rkPassword)
        token = getToken(session, account)
        session = login(session, account, password, codeToken, codeResult, token, count=0)
        getData(account, password, session)

In [ ]:
if __name__ == "__main__":
    #若快打码配置
    rkAccount = input("请输入若快账号：")
    rkPassword = input("密码：")
    auto = input("是否自动循环领取鱼币（是请输入0，否请输入1）：")
    accountList = []
    fh = open('区块猫账号.txt','r')
    for line in fh.readlines():
        linestr = line.strip()
        listlinestr = linestr.split('----')
        accountList.append(listlinestr)
    fh.close()
    if auto=='0':
        delayTime = input("请输入每轮循环延迟时间，单位为秒：")
        print("进入循环领取模式")
        roundtime=0
        while(1):
            roundtime+=1
            print("\n\n         第"+str(roundtime)+"轮循环\n")
            writeLog("\n\n         第"+str(roundtime)+"轮循环\n")
            for i in accountList:
                account = i[0]
                password = i[1]
                mainControl(account, password)
            timeCount = 0
            while(timeCount<int(delayTime)):
                time.sleep(5)
                timeCount+=5
    else:
        for i in accountList:
                account = i[0]
                password = i[1]
                mainControl(account, password)